In [1]:
import data
import model
import numpy as np
from random import choice, seed
from model import build_model, dense, dropout, prelu

Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [3]:
X, y, _, _, _ = data.load("~/Documents/flavours-of-physics-start/input/training.csv",
                          shuffle=True)
Xa, ya, wa, _, _ = data.load("~/Documents/flavours-of-physics-start/input/check_agreement.csv",
                             shuffle=False, weight=True)
Xc, _, _, mc, _ = data.load("~/Documents/flavours-of-physics-start/input/check_correlation.csv",
                            shuffle=False, mass=True, test=True)

X, scaler = data.preprocess_data(X)
Xa, _ = data.preprocess_data(Xa)
Xc, _ = data.preprocess_data(Xc)

X_test, _, _, _, ids = data.load("~/Documents/flavours-of-physics-start/input/test.csv", test=True,
                                 ids=True)
X_test = scaler.transform(X_test)
print "Done"

Done


In [4]:
def create_model(input_size, l=0.5, domain_loss_weight=0.2):

    model = build_model(input_size,
                        [dense(2 * input_size), prelu(),
                         dense(150), prelu(),
                         dense(100), prelu(),
                         dense(75), prelu(),
                         dense(50), prelu(),
                         dense(2)],
                        [dense(200), prelu(),
                         dense(75), prelu(),
                         dense(2)], 0, l, domain_loss_weight=domain_loss_weight)
    return model

In [6]:
np.random.seed(1337)  # repeatability
seed(1337)

n_models = 1
n_epochs = 60
probs = None
for i in range(n_models):
    l = 0
    domain_weight = 1
    print "Model {}; l = {}".format(i, l)
    m = create_model(X.shape[1], l, domain_weight)
    model.fit_model(m, X, y, Xa, ya, wa, Xc, mc, epoch_count=n_epochs, batch_size=64,
                    validation_split=0.05, verbose=2)
    p = model.predict_model(m, np.array(X_test))
    probs = p if probs is None else p + probs
probs /= n_models

Model 0; l = 0
Train on 64175 samples, validate on 3378 samples
Epoch 1/60
KS: 0.19058583868 : 0.09 / CvM: 0.00102265840988 : 0.002
7s - loss: 0.6027 - val_loss: 0.5254
Epoch 2/60
KS: 0.204936614725 : 0.09 / CvM: 0.00108446689826 : 0.002
8s - loss: 0.5497 - val_loss: 0.5307
Epoch 3/60
KS: 0.200301664822 : 0.09 / CvM: 0.00106314089263 : 0.002
7s - loss: 0.5342 - val_loss: 0.5213
Epoch 4/60
KS: 0.22563312933 : 0.09 / CvM: 0.00101482681957 : 0.002
8s - loss: 0.5213 - val_loss: 0.5082
Epoch 5/60
KS: 0.220055492201 : 0.09 / CvM: 0.00106262725509 : 0.002
7s - loss: 0.5136 - val_loss: 0.5011
Epoch 6/60
KS: 0.232362263016 : 0.09 / CvM: 0.000983582368894 : 0.002
7s - loss: 0.5053 - val_loss: 0.5211
Epoch 7/60
KS: 0.221204249701 : 0.09 / CvM: 0.00110032885099 : 0.002
7s - loss: 0.4968 - val_loss: 0.5199
Epoch 8/60
KS: 0.221796701095 : 0.09 / CvM: 0.00108199974322 : 0.002
7s - loss: 0.4907 - val_loss: 0.5073
Epoch 9/60
KS: 0.22391150614 : 0.09 / CvM: 0.00101377300887 : 0.002
10s - loss: 0.4847 - 

In [ ]:
data.save_submission(ids, probs, "grl_prediction.csv")